In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from geopy.distance import great_circle

In [2]:
data=pd.read_excel('Kota_tangerang_result.xlsx')
data

,Salesman,Day,Pattern,ID Outlet,Channel,Latitude,Longitude,Kota,Distance,Group
0,42,Sat,Odd,601642287,Retail,-6.20997,106.608,Kota Tangerang,0.081736,0
1,41,Sat,Odd,601642304,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0
2,41,Sat,Odd,602748094,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0
3,42,Thurs,Odd,601642240,Retail,-6.20942,106.610,Kota Tangerang,0.159372,0
4,41,Sat,Odd,602748096,Retail,-6.20812,106.609,Kota Tangerang,0.159914,0
...,...,...,...,...,...,...,...,...,...,...
19334,1,Sat,Odd,601692768,Retail,-6.03473,106.381,Kota Tangerang,31.790049,41
19335,1,Sat,Odd,601692767,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41
19336,1,Sat,Odd,601692769,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41
19337,1,Sat,Odd,601692828,Retail,-6.03417,106.381,Kota Tangerang,31.828162,41


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19339 entries, 0 to 19338
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Salesman   19339 non-null  int64  
 1   Day        19339 non-null  object 
 2   Pattern    19339 non-null  object 
 3   ID Outlet  19339 non-null  int64  
 4   Channel    19339 non-null  object 
 5   Latitude   19339 non-null  float64
 6   Longitude  19339 non-null  float64
 7   Kota       19339 non-null  object 
 8   Distance   19339 non-null  float64
 9   Group      19339 non-null  int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 1.5+ MB


### `CLUSTERING Hari`

In [4]:
# Fungsi untuk menghitung jarak
def calculate_distance(lat1, lon1, lat2, lon2):
    return great_circle((lat1, lon1), (lat2, lon2)).km

# Fungsi untuk menyesuaikan kelompok
def adjust_groups(data, group_counts, max_group_size, num_groups):
    adjusted_groups = data.copy()
    duplicate_ids = data[data.duplicated(['ID Outlet'], keep=False)]['ID Outlet'].unique()
    
    for outlet_id in duplicate_ids:
        subset = data[data['ID Outlet'] == outlet_id]
        
        prev_group = None
        for index, row in subset.iterrows():
            current_group = row['Hari']
            if prev_group is not None and prev_group != current_group:
                if group_counts.get(prev_group, 0) > max_group_size:
                    adjusted_groups.loc[index, 'Hari'] = (current_group + 1) % num_groups
                else:
                    adjusted_groups.loc[index, 'Hari'] = prev_group
            prev_group = adjusted_groups.loc[index, 'Hari']
    
    return adjusted_groups

# Inisialisasi DataFrame yang akan menampung hasil akhir
all_data_adjusted = pd.DataFrame()

# Definisikan titik pusat
center_lat = -6.20949
center_lon = 106.60856

# Loop untuk setiap kelompok
for group_number in range(42):
    # Filter data untuk kelompok saat ini
    data_group = data[data['Group'] == group_number].copy()  # Pastikan menggunakan .copy()
    
    # Hitung jarak dari titik pusat
    data_group['Distance'] = data_group.apply(lambda row: calculate_distance(center_lat, center_lon, row['Latitude'], row['Longitude']), axis=1)
    
    # Urutkan data berdasarkan jarak
    data_group_sorted = data_group.sort_values(by='Distance').reset_index(drop=True)
    
    # Tentukan jumlah subkelompok dan ukuran setiap subkelompok
    num_subgroups = 6
    total_rows = len(data_group_sorted)
    group_size = total_rows // num_subgroups
    remainder = total_rows % num_subgroups
    
    # Inisialisasi kolom Hari dengan nilai default
    data_group_sorted['Hari'] = np.zeros(total_rows, dtype=int)
    
    # Tentukan batas untuk setiap subkelompok
    boundaries = [i * group_size + min(i, remainder) for i in range(num_subgroups + 1)]
    
    # Alokasikan setiap baris ke subkelompok yang sesuai
    sub_index = 0
    for i in range(total_rows):
        if i >= boundaries[sub_index + 1]:
            sub_index += 1
        data_group_sorted.loc[i, 'Hari'] = sub_index
    
    # Penyesuaian untuk ID Outlet yang sama
    group_counts = data_group_sorted['Hari'].value_counts().sort_index()
    max_group_size = group_size + 1 if remainder > 0 else group_size

    # Terapkan penyesuaian
    data_group_adjusted = adjust_groups(data_group_sorted, group_counts, max_group_size, num_subgroups)
    
    # Tambahkan hasil akhir ke DataFrame utama
    all_data_adjusted = pd.concat([all_data_adjusted, data_group_adjusted], ignore_index=True)
all_data_adjusted = all_data_adjusted.rename(columns={'Group': 'SalesmanNew'})

all_data_adjusted

,Salesman,Day,Pattern,ID Outlet,Channel,Latitude,Longitude,Kota,Distance,SalesmanNew,Hari
0,42,Sat,Odd,601642287,Retail,-6.20997,106.608,Kota Tangerang,0.081736,0,0
1,41,Sat,Odd,601642304,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0,0
2,41,Sat,Odd,602748094,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0,0
3,42,Thurs,Odd,601642240,Retail,-6.20942,106.610,Kota Tangerang,0.159372,0,0
4,41,Sat,Odd,602748096,Retail,-6.20812,106.609,Kota Tangerang,0.159914,0,0
...,...,...,...,...,...,...,...,...,...,...,...
19334,1,Sat,Odd,601692768,Retail,-6.03473,106.381,Kota Tangerang,31.790049,41,5
19335,1,Sat,Odd,601692769,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41,5
19336,1,Sat,Odd,601692767,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41,5
19337,1,Sat,Odd,601692828,Retail,-6.03417,106.381,Kota Tangerang,31.828162,41,5


In [5]:
all_data_adjusted[all_data_adjusted['Channel']=='Grosir']

,Salesman,Day,Pattern,ID Outlet,Channel,Latitude,Longitude,Kota,Distance,SalesmanNew,Hari
618,37,Tues,Even,601656391,Grosir,-6.19636,106.614,Kota Tangerang,1.578990,1,2
620,37,Tues,Odd,601656391,Grosir,-6.19636,106.614,Kota Tangerang,1.578990,1,2
990,33,Wed,Odd,601656390,Grosir,-6.18837,106.615,Kota Tangerang,2.453974,2,0
991,33,Wed,Even,601656390,Grosir,-6.18837,106.615,Kota Tangerang,2.453974,2,0
3523,26,Fri,Odd,600453620,Grosir,-6.17674,106.577,Kota Tangerang,5.043165,7,3
3524,26,Fri,Even,600453620,Grosir,-6.17674,106.577,Kota Tangerang,5.043165,7,3
4471,31,Wed,Even,601683003,Grosir,-6.18577,106.657,Kota Tangerang,5.969141,9,4
4527,31,Wed,Odd,601683003,Grosir,-6.18577,106.657,Kota Tangerang,5.969141,9,4
4656,30,Fri,Odd,602055916,Grosir,-6.18574,106.657,Kota Tangerang,5.970615,10,0
4658,30,Fri,Even,602055916,Grosir,-6.18574,106.657,Kota Tangerang,5.970615,10,0


### `CLUSTERING PATTERN`

In [6]:
# Fungsi untuk menghitung jarak
def calculate_distance(lat1, lon1, lat2, lon2):
    return great_circle((lat1, lon1), (lat2, lon2)).km

# Fungsi untuk membagi data ke dalam grup dengan pengecekan ID Outlet
def assign_groups(data_sorted):
    used_ids = set()  # Set untuk menyimpan ID Outlet yang sudah digunakan
    for index, row in data_sorted.iterrows():
        outlet_id = row['ID Outlet']
        if outlet_id not in used_ids:
            # Jika ID Outlet belum digunakan, alokasikan ke grup A atau B
            if len(used_ids) % 2 == 0:
                data_sorted.loc[index, 'Group'] = 'A'
            else:
                data_sorted.loc[index, 'Group'] = 'B'
            used_ids.add(outlet_id)
        else:
            # Jika ID Outlet sudah digunakan, alokasikan ke grup yang lain
            current_group = data_sorted[data_sorted['ID Outlet'] == outlet_id]['Group'].iloc[0]
            if current_group == 'A':
                data_sorted.loc[index, 'Group'] = 'B'
            else:
                data_sorted.loc[index, 'Group'] = 'A'
    return data_sorted

# Loop untuk setiap kombinasi SalesmanNew dan Hari
all_results = []

for salesman_new in range(42):  # jumlah sales
    for hari in range(6):  # jumlah hari kerja
        # Filter data berdasarkan kombinasi SalesmanNew dan Hari
        data_pattern = all_data_adjusted[(all_data_adjusted['SalesmanNew'] == salesman_new) & (all_data_adjusted['Hari'] == hari)].copy()
        
        if not data_pattern.empty:
            # Hitung jarak dari titik pusat
            data_pattern.loc[:, 'Distance'] = data_pattern.apply(lambda row: calculate_distance(center_lat, center_lon, row['Latitude'], row['Longitude']), axis=1)

            # Urutkan data berdasarkan jarak
            data_pattern_sorted = data_pattern.sort_values(by='Distance').reset_index(drop=True)

            # Inisialisasi kolom Grup dengan tipe data yang sesuai
            data_pattern_sorted['Group'] = pd.Series(dtype='str')

            # Terapkan fungsi untuk alokasi grup
            result = assign_groups(data_pattern_sorted)
            
            # Tambahkan hasil ke daftar
            all_results.append(result)
# Gabungkan semua hasil
final_result = pd.concat(all_results, ignore_index=True)
# Tampilkan hasil akhir
final_result 

,Salesman,Day,Pattern,ID Outlet,Channel,Latitude,Longitude,Kota,Distance,SalesmanNew,Hari,Group
0,42,Sat,Odd,601642287,Retail,-6.20997,106.608,Kota Tangerang,0.081736,0,0,A
1,41,Sat,Odd,601642304,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0,0,B
2,41,Sat,Odd,602748094,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0,0,A
3,42,Thurs,Odd,601642240,Retail,-6.20942,106.610,Kota Tangerang,0.159372,0,0,B
4,41,Sat,Odd,602748096,Retail,-6.20812,106.609,Kota Tangerang,0.159914,0,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
19334,1,Sat,Odd,601692768,Retail,-6.03473,106.381,Kota Tangerang,31.790049,41,5,B
19335,1,Sat,Odd,601692769,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41,5,A
19336,1,Sat,Odd,601692767,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41,5,B
19337,1,Sat,Odd,601692828,Retail,-6.03417,106.381,Kota Tangerang,31.828162,41,5,A


In [7]:
final_result[final_result['Channel']=='Grosir']

,Salesman,Day,Pattern,ID Outlet,Channel,Latitude,Longitude,Kota,Distance,SalesmanNew,Hari,Group
618,37,Tues,Even,601656391,Grosir,-6.19636,106.614,Kota Tangerang,1.578990,1,2,B
620,37,Tues,Odd,601656391,Grosir,-6.19636,106.614,Kota Tangerang,1.578990,1,2,A
990,33,Wed,Odd,601656390,Grosir,-6.18837,106.615,Kota Tangerang,2.453974,2,0,A
991,33,Wed,Even,601656390,Grosir,-6.18837,106.615,Kota Tangerang,2.453974,2,0,B
3523,26,Fri,Odd,600453620,Grosir,-6.17674,106.577,Kota Tangerang,5.043165,7,3,B
3524,26,Fri,Even,600453620,Grosir,-6.17674,106.577,Kota Tangerang,5.043165,7,3,A
4471,31,Wed,Even,601683003,Grosir,-6.18577,106.657,Kota Tangerang,5.969141,9,4,A
4527,31,Wed,Odd,601683003,Grosir,-6.18577,106.657,Kota Tangerang,5.969141,9,4,B
4656,30,Fri,Odd,602055916,Grosir,-6.18574,106.657,Kota Tangerang,5.970615,10,0,A
4658,30,Fri,Even,602055916,Grosir,-6.18574,106.657,Kota Tangerang,5.970615,10,0,B


In [8]:
# Ganti nilai 'Group'
final_result['Group'] = final_result['Group'].replace({'A': 'Odd', 'B': 'Even'})

# Ganti nilai 'Hari'
day_mapping = {
    0: 'Mon',
    1: 'Tues',
    2: 'Wed',
    3: 'Thurs',
    4: 'Fri',
    5: 'Sat'
}
final_result['Hari'] = final_result['Hari'].map(day_mapping)

# Tampilkan DataFrame setelah perubahan
print("\nSetelah perubahan:")
final_result 


Setelah perubahan:


,Salesman,Day,Pattern,ID Outlet,Channel,Latitude,Longitude,Kota,Distance,SalesmanNew,Hari,Group
0,42,Sat,Odd,601642287,Retail,-6.20997,106.608,Kota Tangerang,0.081736,0,Mon,Odd
1,41,Sat,Odd,601642304,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0,Mon,Even
2,41,Sat,Odd,602748094,Retail,-6.20819,106.609,Kota Tangerang,0.152517,0,Mon,Odd
3,42,Thurs,Odd,601642240,Retail,-6.20942,106.610,Kota Tangerang,0.159372,0,Mon,Even
4,41,Sat,Odd,602748096,Retail,-6.20812,106.609,Kota Tangerang,0.159914,0,Mon,Odd
...,...,...,...,...,...,...,...,...,...,...,...,...
19334,1,Sat,Odd,601692768,Retail,-6.03473,106.381,Kota Tangerang,31.790049,41,Sat,Even
19335,1,Sat,Odd,601692769,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41,Sat,Odd
19336,1,Sat,Odd,601692767,Retail,-6.03472,106.381,Kota Tangerang,31.790729,41,Sat,Even
19337,1,Sat,Odd,601692828,Retail,-6.03417,106.381,Kota Tangerang,31.828162,41,Sat,Odd


In [9]:
final_result.to_excel('kota_tang_final_.xlsx', index=False)